# Machine Downtime Prediction - Pipeline Execution

This notebook executes the complete MLOps pipeline for machine downtime prediction.

## Pipeline Steps:
1. **Preprocess Data** - Clean and prepare data
2. **Train Model** - Train RandomForestClassifier
3. **Evaluate Model** - Compute metrics
4. **Register Model** - Register in Model Registry

## 1. Setup and Import Libraries

In [ ]:
import sys
import os
import time
import json
from datetime import datetime

# Add parent directory to path
sys.path.append('..')

import boto3
import sagemaker
from pipeline.pipeline_definition import create_pipeline, upsert_pipeline, start_pipeline_execution

## 2. Initialize SageMaker Session

In [ ]:
# Get SageMaker session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = sagemaker_session.boto_region_name

print(f"SageMaker Role: {role}")
print(f"AWS Region: {region}")
print(f"Default Bucket: {sagemaker_session.default_bucket()}")

## 3. Create/Update Pipeline

In [ ]:
# Create pipeline
print("Creating pipeline...")
pipeline = create_pipeline(config_path='../config/pipeline_config.yaml')
print(f"\nPipeline '{pipeline.name}' created with {len(pipeline.steps)} steps")

## 4. View Pipeline Definition

In [ ]:
# View pipeline definition
pipeline_definition = json.loads(pipeline.definition())

print("Pipeline Steps:")
for step in pipeline_definition['Steps']:
    print(f"  - {step['Name']} ({step['Type']})")

## 5. Upsert Pipeline (Create or Update)

In [ ]:
# Upsert pipeline
print("Upserting pipeline...")
upsert_response = upsert_pipeline(pipeline)
print(f"\nPipeline ARN: {upsert_response['PipelineArn']}")
print("✓ Pipeline created/updated successfully!")

## 6. Start Pipeline Execution

In [ ]:
# Start execution
execution_display_name = f"Execution-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
print(f"Starting pipeline execution: {execution_display_name}")

execution = start_pipeline_execution(pipeline, execution_display_name)
print(f"\nExecution ARN: {execution.arn}")
print("\n✓ Pipeline execution started!")

## 7. Monitor Pipeline Execution

In [ ]:
# Monitor execution
print("Monitoring pipeline execution...\n")
print("Steps Status:")
print("-" * 80)

execution.wait(delay=30, max_attempts=60)

# Get execution details
execution_details = execution.describe()
print(f"\nExecution Status: {execution_details['PipelineExecutionStatus']}")
print(f"Start Time: {execution_details['CreationTime']}")

if 'EndTime' in execution_details:
    print(f"End Time: {execution_details['EndTime']}")
    duration = (execution_details['EndTime'] - execution_details['CreationTime']).total_seconds() / 60
    print(f"Duration: {duration:.2f} minutes")

## 8. Get Step Details

In [ ]:
# List execution steps
steps = execution.list_steps()

print("\nPipeline Steps Summary:")
print("=" * 80)

for step in steps:
    print(f"\nStep: {step['StepName']}")
    print(f"  Status: {step['StepStatus']}")
    print(f"  Start Time: {step.get('StartTime', 'N/A')}")
    print(f"  End Time: {step.get('EndTime', 'N/A')}")

## 9. View Model Registry

In [ ]:
# Get SageMaker client
sm_client = boto3.client('sagemaker', region_name=region)

# List model packages
model_package_group_name = 'machine-downtime-model-group'

try:
    response = sm_client.list_model_packages(
        ModelPackageGroupName=model_package_group_name,
        SortBy='CreationTime',
        SortOrder='Descending',
        MaxResults=5
    )
    
    print(f"\nRegistered Models in '{model_package_group_name}':")
    print("=" * 80)
    
    if response['ModelPackageSummaryList']:
        for idx, package in enumerate(response['ModelPackageSummaryList'], 1):
            print(f"\nModel {idx}:")
            print(f"  ARN: {package['ModelPackageArn']}")
            print(f"  Status: {package['ModelApprovalStatus']}")
            print(f"  Created: {package['CreationTime']}")
    else:
        print("No model packages found.")
        
except Exception as e:
    print(f"Error listing model packages: {str(e)}")

## 10. Summary

In [ ]:
print("="*80)
print("PIPELINE EXECUTION COMPLETE")
print("="*80)
print(f"Pipeline Name: {pipeline.name}")
print(f"Execution ARN: {execution.arn}")
print(f"Status: {execution_details['PipelineExecutionStatus']}")
print("="*80)